In [1]:
!pip3 install labelme tensorflow-macos opencv-python matplotlib albumentations

In [2]:
import os
import time
import uuid
import cv2

In [20]:
IMAGES_PATH = os.path.join('/Users/veerchheda/coding/DL/faceDetection/data/images')
number_images = 30

In [21]:
if not os.path.exists(IMAGES_PATH):
    os.makedirs(IMAGES_PATH)

In [17]:
cap = cv2.VideoCapture(0)  # Initialize camera (change the index if needed)

if not cap.isOpened():
    print("Error: Failed to open camera.")
    exit()

while True:
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to capture frame.")
        break

    imgname = os.path.join(IMAGES_PATH, f'{str(uuid.uuid1())}.jpg')

    if cv2.imwrite(imgname, frame):
        print(f"Image saved as: {imgname}")
    else:
        print("Error: Failed to write image.")

    cv2.imshow('frame', frame)
    time.sleep(0.5)  # Adjust delay as needed

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Image saved as: /Users/veerchheda/coding/DL/faceDetection/data/images/3597e31c-ede1-11ee-8be1-4efb21a06e4f.jpg


2024-03-29 21:00:02.426 Python[83318:3119604] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


Image saved as: /Users/veerchheda/coding/DL/faceDetection/data/images/36041aa0-ede1-11ee-8be1-4efb21a06e4f.jpg
Image saved as: /Users/veerchheda/coding/DL/faceDetection/data/images/36550dca-ede1-11ee-8be1-4efb21a06e4f.jpg
Image saved as: /Users/veerchheda/coding/DL/faceDetection/data/images/36a66558-ede1-11ee-8be1-4efb21a06e4f.jpg
Image saved as: /Users/veerchheda/coding/DL/faceDetection/data/images/36f7b8c2-ede1-11ee-8be1-4efb21a06e4f.jpg
Image saved as: /Users/veerchheda/coding/DL/faceDetection/data/images/374ee61a-ede1-11ee-8be1-4efb21a06e4f.jpg
Image saved as: /Users/veerchheda/coding/DL/faceDetection/data/images/37a06d96-ede1-11ee-8be1-4efb21a06e4f.jpg
Image saved as: /Users/veerchheda/coding/DL/faceDetection/data/images/37f1b200-ede1-11ee-8be1-4efb21a06e4f.jpg
Image saved as: /Users/veerchheda/coding/DL/faceDetection/data/images/38432ee6-ede1-11ee-8be1-4efb21a06e4f.jpg
Image saved as: /Users/veerchheda/coding/DL/faceDetection/data/images/3894d1d8-ede1-11ee-8be1-4efb21a06e4f.jpg
I

In [2]:
!labelme

/Users/veerchheda/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2024-03-29 21:23:26,576 [INFO   ] __init__:get_config:67- Loading config file from: /Users/veerchheda/.labelmerc
2024-03-29 21:23:26.995 Python[84011:3144370] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
Corrupt JPEG data: 4 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


In [3]:
import json
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

In [4]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0']


In [37]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [38]:
import os
os.environ["TF_DISABLE_MLC"] = "1"
os.environ["TF_MLC_LOGGING"] = "1"
import tensorflow as tf
from tensorflow.python.compiler. import mlcompute

tf.compat.v1.disable_eager_execution()
mlcompute.set_mlc_device(device_name='gpu')
print("is_apple_mlc_enabled %s" % mlcompute.is_apple_mlc_enabled())
print("is_tf_compiled_with_apple_mlc %s" % mlcompute.is_tf_compiled_with_apple_mlc())
print(f"eagerly? {tf.executing_eagerly()}")
print(tf.config.list_logical_devices())

ImportError: cannot import name 'mlcompute' from 'tensorflow.python.compiler.tensorrt' (/Users/veerchheda/Library/Python/3.9/lib/python/site-packages/tensorflow/python/compiler/tensorrt/__init__.py)

In [6]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [7]:
images = tf.data.Dataset.list_files('./data/images/*.jpg')

In [8]:
images.as_numpy_iterator().next()

b'./data/images/42407ca0-ede1-11ee-8be1-4efb21a06e4f.jpg'

In [9]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [10]:
images = images.map(load_image)

In [4]:
from tensorflow.keras.models import load_model

In [5]:
facetracker = load_model('/Users/veerchheda/coding/DL/faceDetection/facetracker.h5', safe_mode=True)

In [27]:
!pip3 install tensorflow==2.15.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.15.0-cp39-cp39-macosx_12_0_arm64.whl.metadata (3.6 kB)
Using cached tensorflow-2.15.0-cp39-cp39-macosx_12_0_arm64.whl (2.1 kB)


In [6]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [41]:
import cv2
import numpy as np
import tensorflow as tf

# Initialize the video capture object
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # Read frame from video capture
    ret, frame = cap.read()
    
    if not ret:
        break

    # Adjust the cropping region to make the frame larger
    frame = frame[10:690, 50:1230, :]

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120, 120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [1180, 680]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [1180, 680]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [1180, 680]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [1180, 680]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [1180, 680]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 42ms/step


: 